In [6]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
import re
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
import socket
import numpy as np
import threading
import io
import torch
import soundfile as sf
import datetime
import os
# import main.assistants.modules.LLM as LLM

model_id = "openai/whisper-large-v3"


RuntimeError: asyncio.run() cannot be called from a running event loop

In [7]:
def get_whisper(model):
    torch_dtype = torch.float16 if model == "whisper" else torch.float32

    if model == "whisper":
        model_id = "openai/whisper-large-v3"
    if model == "d_whisper":
        model_id = "distil-whisper/distil-large-v3"

    return model_id, torch_dtype

device = "cuda:0" if torch.cuda.is_available() else "cpu"

processor = AutoProcessor.from_pretrained(model_id)
model_id, torch_dtype = get_whisper("whisper")

whisper = AutoModelForSpeechSeq2Seq.from_pretrained(model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True)
whisper.to(device)
filtered_phrases = ["감사합니다.", "네.", "감사합니다", ".", "네", "아멘"]



def do_whisper(audio_buffer, dtype):
    audio, sample_rate = sf.read(audio_buffer, dtype='float32')
    inputs = processor(audio, sampling_rate=sample_rate, return_tensors="pt")
    input_features = inputs.input_features.to(device, dtype=dtype) 
    gen_kwargs = {
        "max_new_tokens": 50,
        "num_beams": 1,
        "do_sample": True,
        "temperature": 0.4,
        "return_timestamps": False,
        "language": "korean"
    }

    # Whisper process
    with torch.no_grad():
        pred_ids = whisper.generate(input_features, **gen_kwargs)
        user_text = processor.batch_decode(pred_ids, skip_special_tokens=True)
        print(user_text)
        print(user_text[0])

    if user_text in filtered_phrases:
        print(f"Filtered phrase detected: {user_text}")
        pass

    if user_text:
        print(f"Sending user_text: {user_text}")
    else:
        user_text = "."

    return user_text

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
while True:    
    timestamp = datetime.datetime.now().strftime("%m%d%H%M%S")
            
            # 로컬 오디오 파일 읽기
    data, samplerate = sf.read("files/spoken_240705042136.wav")
    
    # BytesIO 객체에 오디오 데이터 저장
    audio_buffer = io.BytesIO()
    sf.write(audio_buffer, data, samplerate, format='WAV')
    audio_buffer.seek(0)
    
    # 오디오 데이터를 WAV 파일로 저장 (테스트용)
    received_wav_file_path = f"files/received_audio_{timestamp}.wav"
    with open(received_wav_file_path, 'wb') as f:
        f.write(audio_buffer.read())
    print(f"Audio saved to {received_wav_file_path}")

    # Whisper process
    audio_buffer.seek(0)  # Whisper 처리를 위해 다시 스트림의 시작 위치로 이동
    user_text = do_whisper(audio_buffer, dtype=torch_dtype)

    # LLM process
    # gpt_text = LLM.GPT(user_text)

    print({"timestamp": {"user_text": user_text}})



Audio saved to files/received_audio_0705082608.wav


c:\Users\hoon1\anaconda3\envs\torch-latest\lib\site-packages\transformers\models\whisper\modeling_whisper.py:691: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


[' 안녕']
 안녕
Sending user_text: [' 안녕']
{'timestamp': {'user_text': [' 안녕']}}
Audio saved to files/received_audio_0705082609.wav
[' 안녕하세요']
 안녕하세요
Sending user_text: [' 안녕하세요']
{'timestamp': {'user_text': [' 안녕하세요']}}
Audio saved to files/received_audio_0705082609.wav
[' 안녕']
 안녕
Sending user_text: [' 안녕']
{'timestamp': {'user_text': [' 안녕']}}
Audio saved to files/received_audio_0705082609.wav
[' 안녕하세요']
 안녕하세요
Sending user_text: [' 안녕하세요']
{'timestamp': {'user_text': [' 안녕하세요']}}
Audio saved to files/received_audio_0705082610.wav
[' 안녕']
 안녕
Sending user_text: [' 안녕']
{'timestamp': {'user_text': [' 안녕']}}
Audio saved to files/received_audio_0705082610.wav
[' 안녕']
 안녕
Sending user_text: [' 안녕']
{'timestamp': {'user_text': [' 안녕']}}
Audio saved to files/received_audio_0705082610.wav
[' 안녕']
 안녕
Sending user_text: [' 안녕']
{'timestamp': {'user_text': [' 안녕']}}
Audio saved to files/received_audio_0705082610.wav
[' 안녕']
 안녕
Sending user_text: [' 안녕']
{'timestamp': {'user_text': [' 안녕']}}
Aud

KeyboardInterrupt: 